In [ ]:
# =============================================================================
# XY 좌표 보간 모듈 (coordinate_interpolation_module.ipynb)
# 작성자: [김병현]
# 생성일: 2025-07-16
# 설명: 도로명+아파트명 매칭을 통해 XY_coordinates.csv로 좌표 결측치를 보간하는 모듈
# =============================================================================

def apply_optimized_merge_coordinates(concat, xy_coords_path):
    """
    최적화된 pandas merge를 이용한 고속 좌표 보간 함수
    
    Parameters:
    -----------
    concat : pandas.DataFrame
        train+test 합쳐진 데이터프레임
    xy_coords_path : str
        XY_coordinates.csv 파일 경로
        
    Returns:
    --------
    pandas.DataFrame
        좌표가 보간된 데이터프레임
    """
    print("\n" + "="*60)
    print("최적화된 pandas merge 좌표 보간 시작")
    print("="*60)
    
    # XY_coordinates.csv 로드 (필요 컬럼만) - 메모리 절약을 위해 usecols 사용
    try:
        xy_coords = pd.read_csv(
            xy_coords_path,
            usecols=['도로명', '아파트명', '좌표X', '좌표Y']
        )
        print(f"XY 좌표 데이터 로드 완료: {xy_coords.shape}")
    except Exception as e:
        print(f"파일 로드 실패: {e}")
        return concat
    
    # 결측·중복 제거 및 문자열 정규화
    xy_coords = xy_coords.dropna(subset=['도로명', '아파트명', '좌표X', '좌표Y'])
    xy_coords['도로명'] = (
        xy_coords['도로명'].astype(str)
                 .str.strip().str.lower()
    )
    xy_coords['아파트명'] = (
        xy_coords['아파트명'].astype(str)
                 .str.strip().str.lower()
    )
    # 도로명+아파트명 그룹별 평균 좌표 취합
    xy_coords = (
        xy_coords.groupby(['도로명', '아파트명'])
                 .agg({'좌표X': 'mean', '좌표Y': 'mean'})
                 .reset_index()
    )
    print(f"전처리 완료: {xy_coords.shape}")
    
    # 원본 데이터 복사
    concat_result = concat.copy()
    
    # 도로명, 아파트명 컬럼 자동 탐색
    road_cols = [c for c in concat_result.columns if '도로' in c]
    apt_cols = [c for c in concat_result.columns if '아파트' in c]
    if not road_cols or not apt_cols:
        print("경고: 도로명 또는 아파트명 컬럼을 찾을 수 없습니다.")
        return concat
    road_col = road_cols[0]
    apt_col = apt_cols[0]
    print(f"사용할 컬럼: 도로명='{road_col}', 아파트명='{apt_col}'")
    
    # 문자열 정규화를 위한 임시 컬럼 추가
    concat_result[f"{road_col}_clean"] = (
        concat_result[road_col].astype(str)
                     .str.strip().str.lower()
    )
    concat_result[f"{apt_col}_clean"] = (
        concat_result[apt_col].astype(str)
                     .str.strip().str.lower()
    )
    
    # 결측 통계 출력
    total_rows = len(concat_result)
    missing_x = concat_result['좌표X'].isnull().sum()
    missing_y = concat_result['좌표Y'].isnull().sum()
    print(f"기존 좌표 결측치: X={missing_x:,}개 ({missing_x/total_rows*100:.1f}%), Y={missing_y:,}개 ({missing_y/total_rows*100:.1f}%)")
    
    # pandas merge 수행
    print("고속 병합 중...")
    merged = concat_result.merge(
        xy_coords,
        left_on=[f"{road_col}_clean", f"{apt_col}_clean"],
        right_on=['도로명', '아파트명'],
        how='left',
        suffixes=('', '_new')
    )
    # 여기서 merged의 인덱스가 concat_result와 달라질 수 있으므로,
    # concat_result와 동일한 인덱스로 재설정해 줍니다.
    merged = merged.reindex(concat_result.index)
    
    matched = merged['좌표X_new'].notna().sum()
    print(f"매칭 성공: {matched:,}개")
    
    # 결측치 보간
    if matched > 0:
        # 기존 결측인 위치만 업데이트하도록 Boolean mask 작성
        x_mask = concat_result['좌표X'].isnull() & merged['좌표X_new'].notna()
        concat_result.loc[x_mask, '좌표X'] = merged.loc[x_mask, '좌표X_new']
        y_mask = concat_result['좌표Y'].isnull() & merged['좌표Y_new'].notna()
        concat_result.loc[y_mask, '좌표Y'] = merged.loc[y_mask, '좌표Y_new']
    
    # 임시 컬럼 제거
    concat_result = concat_result.drop([
        f"{road_col}_clean", f"{apt_col}_clean"
    ], axis=1)
    
    # 결과 통계 출력
    new_missing_x = concat_result['좌표X'].isnull().sum()
    new_missing_y = concat_result['좌표Y'].isnull().sum()
    print(f"최종 좌표 결측치: X {new_missing_x:,}개, Y {new_missing_y:,}개")
    print("최적화된 pandas merge 좌표 보간 완료!")
    print("="*60)
    
    return concat_result


# 모듈 로드 완료 메시지
print("최적화된 pandas merge 좌표 보간 모듈 로드 완료!")
print("사용법:")
print("  concat = apply_optimized_merge_coordinates(concat, 'XY_coordinates.csv')")

# 시각화 함수는 필요시 그대로 사용 가능합니다.  설명 생략
